In [ ]:
%load_ext dotenv
%dotenv
%load_ext mypy_ipython
    

In [ ]:
from langgraph.graph import START, END, StateGraph, add_messages, MessagesState

In [ ]:
from typing_extensions import TypedDict

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

In [ ]:
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage, RemoveMessage, SystemMessage

In [ ]:
from langchain_core.runnables import Runnable

In [ ]:
from collections.abc import Sequence

In [ ]:
from typing import Literal, Annotated

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

In [ ]:
import sqlite3

In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver

In [ ]:
class State(MessagesState):
    summary: str

## Define the Nodes

In [ ]:
chat = ChatOpenAI(model= 'gpt-4', seed = 365, temperature = 0, max_completion_tokens = 100)

In [ ]:
def ask_question(state: State) -> State:
    
    print(f"\n-------> ENTERING ask_question:")
    
    question = "What is your question?"
    print(question)
    
    return State(messages = [AIMessage(question), HumanMessage(input())])

In [ ]:
def summarize_messages(state: State) -> State:
    print(f"\n-------> ENTERING summarize_messages:")

    new_conversation = ""
    for i in state["messages"]:
        new_conversation += f"{i.type}: {i.content}\n\n"
    
    summary_instructions = f'''
Update the ongoing summary by incorporating the new lines of conversation below. 
Build upon the previous summary rather than repeating it, 
so that the result reflects the most recent context and developments.
Respond only with the summary.

Previous Summary:
{state.get("summary", "")}

New Conversation:
{new_conversation}
'''

    print(summary_instructions)

    summary = chat.invoke([HumanMessage(summary_instructions)])
    
    remove_messages = [RemoveMessage(id = i.id) for i in state["messages"][:]]

    return State(messages = remove_messages, summary = summary.content)

In [ ]:
def chatbot(state: State) -> State:
    
    print(f"\n-------> ENTERING chatbot:")
        
    system_message = f'''
    Here's a quick summary of what's been discussed so far:
    {state.get("summary", "")}
    
    Keep this in mind as you answer the next question.
    '''
    
    response = chat.invoke([SystemMessage(system_message)] + state["messages"])
    response.pretty_print()
    
    return State(messages = [response])

## Define the Graph

In [ ]:
graph = StateGraph(State)

In [ ]:
graph.add_node("ask_question", ask_question)
graph.add_node("chatbot", chatbot)

graph.add_node("summarize_messages", summarize_messages)

graph.add_edge(START, "ask_question")
graph.add_edge("ask_question", "chatbot")
graph.add_edge("chatbot", "summarize_messages")
graph.add_edge("summarize_messages", END)


In [ ]:
db_path = "./langgraph.db"
con = sqlite3.connect(database = db_path, check_same_thread = False)

In [ ]:
checkpointer = SqliteSaver(con)

In [ ]:

graph_compiled = graph.compile(checkpointer)

In [ ]:
isinstance(graph_compiled, Runnable)

In [ ]:
graph_compiled

## Test the Graph

In [ ]:
config1 = {"configurable": {"thread_id": "1"}}
config2 = {"configurable": {"thread_id": "2"}}

In [ ]:
graph_compiled.invoke(State(), config1)

## To debug what is inside

In [ ]:
graph_states = [i for i in graph_compiled.get_state_history(config1)]

In [ ]:
graph_states

In [ ]:
for i in graph_states[::-1]:
    print(f'''
    Messages: {i.values["messages"]}
    Summary: {i.values.get("summary", "")}
    Next: {i.next}
    Step: {i.metadata["step"]}
    ''')